In [1]:
import logging

logger = logging.getLogger()
logger.disabled = True

In [2]:
from memory.environment.gym import MemoryEnv
from memory import EpisodicMemory, SemanticMemory
from memory.utils import seed_everything
from itertools import count
import numpy as np
import random
from tqdm.notebook import tqdm


def get_diff_episodic(max_step, generator_params, caps, seed):

    seed_everything(seed)
    env = MemoryEnv(max_step=max_step, generator_params=generator_params)

    results = []
    for capacity in caps:
        rewards = {}
        for policy in ["oldest", "random"]:

            rewards[policy] = 0
            M_e = EpisodicMemory(capacity)
            env.reset()

            for t in count():
                if M_e.is_kinda_full:
                    if policy == "random":
                        M_e.forget_random()
                    else:
                        M_e.forget_oldest()

                while not M_e.is_full:
                    ob, _, _, _ = env.step(mode="observation")
                    M_e.add(EpisodicMemory.ob2epi(ob))

                question, _, _, _ = env.step(mode="question")

                pred = M_e.answer_latest(question)

                _, reward, done, _ = env.step(mode="answer", action=pred)

                rewards[policy] += reward

                ob, _, _, _ = env.step(mode="observation")
                M_e.add(EpisodicMemory.ob2epi(ob))
                if done:
                    print(capacity, policy, rewards[policy])
                    break

        results.append((rewards["oldest"] - rewards["random"]) / rewards["oldest"])
        print((rewards["oldest"] - rewards["random"]) / rewards["oldest"])
        print()

    return np.mean(results)


def get_diff_semantic(max_step, generator_params, caps, seed):

    seed_everything(seed)
    env = MemoryEnv(max_step=max_step, generator_params=generator_params)

    results = []
    for capacity in caps:
        rewards = {}
        for policy in ["weakest", "random"]:

            rewards[policy] = 0
            M_s = SemanticMemory(capacity)
            env.reset()

            for t in count():
                if M_s.is_kinda_full:
                    if policy == "random":
                        M_s.forget_random()
                    elif policy == "weakest":
                        M_s.forget_weakest()
                    else:
                        raise ValueError

                while not M_s.is_full:
                    ob, _, _, _ = env.step(mode="observation")
                    M_s.add(SemanticMemory.ob2sem(ob))

                question, _, _, _ = env.step(mode="question")

                pred = M_s.answer_strongest(question)

                _, reward, done, _ = env.step(mode="answer", action=pred)

                rewards[policy] += reward

                ob, _, _, _ = env.step(mode="observation")
                M_s.add(SemanticMemory.ob2sem(ob))
                if done:
                    print(capacity, policy, rewards[policy])
                    break


        results.append((rewards["weakest"] - rewards["random"]) / rewards["weakest"])
        print((rewards["weakest"] - rewards["random"]) / rewards["weakest"])
        print()

    return np.mean(results)


def get_diff_es(max_step, generator_params, caps, seed, pretrain):

    seed_everything(seed)
    env = MemoryEnv(max_step=max_step, generator_params=generator_params)

    results = []
    for capacity in caps:
        rewards = {}
        for policy in ["find_common", "random"]:

            rewards[policy] = 0
            M_e = EpisodicMemory(capacity // 2)
            M_s = SemanticMemory(capacity // 2)

            if pretrain:
                free_space = M_s.pretrain_semantic(generator_params)
                M_e.increase_capacity(free_space)
                assert M_e.capacity + M_s.capacity == capacity

            ob, question = env.reset()
            M_e.add(EpisodicMemory.ob2epi(ob))

            for t in count():
                if M_e.is_kinda_full:
                    if policy == "random":
                        M_e.forget_random()
                    elif policy == "find_common":

                        if not pretrain:
                            mem_epis, mem_sem = M_e.get_similar()
                        else:
                            mem_epis, mem_sem = None, None

                        if mem_epis is not None:
                            M_s.add(mem_sem)
                            if M_s.is_kinda_full:
                                M_s.forget_weakest()
                                assert M_s.is_full
                            for mem_epi_ in mem_epis:
                                M_e.forget(mem_epi_)
                        else:
                            M_e.forget_oldest()
                            assert M_e.is_full

                    else:
                        raise ValueError

                if M_e.is_answerable(question):
                    pred = M_e.answer_latest(question)
                else:
                    pred = M_s.answer_strongest(question)

                (ob, question), reward, done, info = env.step(pred)

                M_e.add(EpisodicMemory.ob2epi(ob))

                rewards[policy] += reward

                if done:
                    print(capacity, policy, rewards[policy])
                    break

        results.append(
            (rewards["find_common"] - rewards["random"]) / rewards["find_common"]
        )
        print((rewards["find_common"] - rewards["random"]) / rewards["find_common"])
        print()

    return np.mean(results)


seed = 42
max_step = 10000
caps = [2, 4, 8, 16, 32]
generator_params = {
    "max_history": 100,
    "semantic_knowledge_path": "./data/semantic-knowledge.json",
    "names_path": "./data/top-human-names",
    "weighting_mode": "highest",
    "commonsense_prob": 0.5,
    "time_start_at": 0,
    "limits": {
        "heads": 10,
        "tails": 1,
        "names": 10,
        "allow_spaces": False,
    },
    "disjoint_entities": True,
}
seed_everything(seed)
env = MemoryEnv(max_step=max_step, generator_params=generator_params)

print(len(env.oqag.heads))
print(len(env.oqag.tails))
print(len(env.oqag.names))


10
7
10


In [7]:
M_e = EpisodicMemory(4)

M_e.add(["Tae's laptop", "AtLocation", "Tae's desk", 0])
M_e.add(["Foo's laptop", "AtLocation", "Foo's desk", 1])
M_e.add(["Bar's laptop", "AtLocation", "Bar's bar", 2])
M_e.add(["Baz's laptop", "AtLocation", "Baz's kitchen", 3])

M_e.get_similar()

> /home/tk/repos/explicit-memory/memory/memory.py(551)get_similar()
    549         import pdb; pdb.set_trace()
    550 
--> 551         if len(semantic_possibles) == len(entries):
    552             logging.info("no episodic memories found to be compressible.")
    553             return None, None

{'laptop^AtLocation^desk': [0, 1], 'laptop^AtLocation^kitchen': [3], 'laptop^AtLocation^bar': [2]}


BdbQuit: 

In [14]:
diff = get_diff_episodic(max_step, generator_params, caps, seed)
diff

2 oldest 611
2 random 595
0.02618657937806874

4 oldest 1183
4 random 1184
-0.0008453085376162299

8 oldest 2233
8 random 2043
0.08508732646663682

16 oldest 4098
16 random 3550
0.13372376769155686

32 oldest 6660
32 random 5452
0.18138138138138138



0.0851067492760055

In [16]:
diff = get_diff_semantic(max_step, generator_params, caps, 40)
diff

2 weakest 1109
2 random 1223
-0.10279531109107304

4 weakest 1367
4 random 2135
-0.5618141916605706

8 weakest 2491
8 random 3276
-0.3151344841429145

16 weakest 4085
16 random 4382
-0.07270501835985312

32 weakest 5624
32 random 4727
0.1594950213371266



-0.17859079678345696

In [31]:
diff = get_diff_es(max_step, generator_params, caps, seed, False)
diff

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
diff = get_diff_es(max_step, generator_params, caps, seed, True)
diff